In [7]:
pip install transformers

     |████████████████████████████████| 3.4 MB 8.6 MB/s 
     |████████████████████████████████| 61 kB 513 kB/s 
     |████████████████████████████████| 895 kB 63.2 MB/s 
     |████████████████████████████████| 3.3 MB 62.4 MB/s 
     |████████████████████████████████| 596 kB 40.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
pip install tflearn

     |████████████████████████████████| 107 kB 12.1 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=d7dfa96cde0c6551a50b249d7f46bf26ef6a2674c4d82556ed593ad90e1ceaa3
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [9]:
import re
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
import tensorflow as tf
import json
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

# HuggingFace NLP library
from transformers import TFBertForSequenceClassification, BertTokenizer


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import tensorflow as tf
nltk.download('popular', quiet=True) # for downloading packages

True

In [10]:
with open('data.json') as fin:
    data = json.load(fin)

In [11]:
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
#Tokenisation
contractions_dict = { "ain’t": "are not", "’s":" is", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "‘cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have", "hasn’t": "has not", "haven’t": "have not", "he’d": "he would", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "I’d": "I would", "I’d’ve": "I would have", "I’ll": "I will", "I’ll’ve": "I will have", "I’m": "I am", "I’ve": "I have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not", "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have", "that’d": "that would", "that’d’ve": "that would have", "there’d": "there would", "there’d’ve": "there would have", "they’d": "they would", "they’d’ve": "they would have","they’ll": "they will",
 "they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not","what’ll": "what will", "what’ll’ve": "what will have", "what’re": "what are", "what’ve": "what have", "when’ve": "when have", "where’d": "where did", "where’ve": "where have", "i'm" : "i am","there's":"there is","i've":"i have","what's":"what is","can't":"can not","don't":"do not","i'll":"i will","we're":"we are","it's":"it is","yr":"year",
 "who’ll": "who will", "who’ll’ve": "who will have", "who’ve": "who have", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have", "you’re": "you are", "you’ve": "you have"}

contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, s)


def preprocess(sentences):
    sentences  = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",sentences).split())
    sentences  = sentences.replace('https?:\/\/.*[\r\n]*', '')
    # sentences  = sentences.normalize('NFKD').encode('ascii', errors='ignore').decode('utf-8')
    sentences  = sentences.replace('\d+', '')
    sentences  = sentences.replace('[^\w\s]', '')
    sentences  = expand_contractions(sentences)
  

In [13]:
words = []
sentences = []
labels = []
docs_x = []
docs_y = []
bert_inputs = []
X = []
y = []
labels = []

In [14]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmer = WordNetLemmatizer()
stemmer = LancasterStemmer()

def StemTokens(tokens):
    return [stemmer.stem(token.lower()) for token in tokens]
def LemTokens(tokens):
    return [lemmer.lemmatize(token.lower()) for token in tokens]
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [15]:
for intent in data['intents']:
      for pattern in intent['patterns']:
            preprocess(pattern)
            tokenize = LemNormalize(pattern)
            words.extend(tokenize)
            sentences.append(pattern)
            docs_x.append(tokenize)
            docs_y.append(intent["tag"])

      labels.append(intent["tag"])


In [16]:
contractions_dict = { "ain’t": "are not", "’s":" is", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "‘cause": "because", "could’ve": "could have", "couldn’t": "could not", "couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have", "hasn’t": "has not", "haven’t": "have not", "he’d": "he would", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "I’d": "I would", "I’d’ve": "I would have", "I’ll": "I will", "I’ll’ve": "I will have", "I’m": "I am", "I’ve": "I have", "isn’t": "is not", "it’d": "it would", "it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "let’s": "let us", "ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not", "mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have", "shan’t": "shall not", "sha’n’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will", "she’ll’ve": "she will have", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have", "so’ve": "so have", "that’d": "that would", "that’d’ve": "that would have", "there’d": "there would", "there’d’ve": "there would have", "they’d": "they would", "they’d’ve": "they would have","they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would", "we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not","what’ll": "what will", "what’ll’ve": "what will have", "what’re": "what are", "what’ve": "what have", "when’ve": "when have", "where’d": "where did", "where’ve": "where have", "i'm" : "i am","there's":"there is","i've":"i have","what's":"what is","can't":"can not","don't":"do not","i'll":"i will","we're":"we are","it's":"it is","yr":"year",
"who’ll": "who will", "who’ll’ve": "who will have", "who’ve": "who have", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have", "would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have", "y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have", "you’re": "you are", "you’ve": "you have"}

contractions_re = re.compile('(%s)'%'|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, s)

In [17]:
words = LemTokens(words)
words = sorted(list(set(words)))

labels = sorted(labels)

X = []
y = []

init = [0]*(len(labels))

for x, doc in enumerate(docs_x):
      bag = []
      find_words = LemTokens(doc)
      
      for w in words:
            if w in find_words:
                  bag.append(1)
            else:
                  bag.append(0)
                  
      output_row = init[:]
      output_row[labels.index(docs_y[x])] = 1
      X.append(bag)
      y.append(output_row)

In [18]:
training = np.array(X)
output = np.array(y)

# Modelling

## with DNN tflearn 

In [ ]:
import tflearn
tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [ ]:
model.fit(training, output, n_epoch=800, batch_size=8, show_metric=True)

Training Step: 3999  | total loss: 0.24502 | time: 0.016s
| Adam | epoch: 800 | loss: 0.24502 - acc: 0.9855 -- iter: 32/35
Training Step: 4000  | total loss: 0.24502 | time: 0.019s
| Adam | epoch: 800 | loss: 0.24502 - acc: 0.9870 -- iter: 35/35
--


In [29]:
def bag_of_words(s, words):
    bag = [0]*len(words)
    s_words = LemNormalize(s)
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array([bag])

In [ ]:
pred = model.predict(bag_of_words('hello',words))

In [ ]:
pred[0]

array([1.1171675e-06, 1.1157947e-02, 2.2766133e-09, 1.1470751e-03,
       1.4294038e-06, 9.5514870e-01, 1.2102793e-09, 2.1294561e-04,
       3.2330740e-02, 5.3988103e-08], dtype=float32)

In [ ]:
labels[np.argmax(pred[0])]

'greeting'

In [ ]:
model.save('model.tflearn')

INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## with DNN Keras

In [19]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("keras_model.h5",save_best_weights=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [27]:
def model(n_layers=2,n_neurons=8,input_shape=[None, len(training[0])]):
      model  = tf.keras.models.Sequential()
      model.add(tf.keras.layers.Input(shape = input_shape))
      for _ in range(n_layers):
        model.add(tf.keras.layers.Dense(n_neurons,kernel_initializer='he_normal',activation='elu'))
      model.add(tf.keras.layers.Dense(len(labels),activation='softmax'))
      model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Nadam(),metrics=['accuracy'])
      return model
    
model = model()

In [28]:
model.fit(training,np.array([np.argmax(x) for x in output]),batch_size=5,epochs=800,callbacks=[checkpoint_cb,early_stopping_cb])

Epoch 1/800
9/9 [==============================] - 1s 5ms/step - loss: 2.4681 - accuracy: 0.0227
Epoch 2/800
9/9 [==============================] - 0s 4ms/step - loss: 2.4447 - accuracy: 0.0227
Epoch 3/800
9/9 [==============================] - 0s 4ms/step - loss: 2.4173 - accuracy: 0.0682
Epoch 4/800
9/9 [==============================] - 0s 5ms/step - loss: 2.3904 - accuracy: 0.0682
Epoch 5/800
9/9 [==============================] - 0s 4ms/step - loss: 2.3623 - accuracy: 0.0909
Epoch 6/800
9/9 [==============================] - 0s 5ms/step - loss: 2.3355 - accuracy: 0.1591
Epoch 7/800
9/9 [==============================] - 0s 6ms/step - loss: 2.3089 - accuracy: 0.2045
Epoch 8/800
9/9 [==============================] - 0s 5ms/step - loss: 2.2809 - accuracy: 0.2045
Epoch 9/800
9/9 [==============================] - 0s 4ms/step - loss: 2.2541 - accuracy: 0.2273
Epoch 10/800
9/9 [==============================] - 0s 4ms/step - loss: 2.2272 - accuracy: 0.2955
Epoch 11/800
9/9 [===========

In [32]:
pred = model.predict(bag_of_words('whats your name',words))

In [33]:
labels[np.argmax(pred[0])]

'name'